In [1]:
from pocketsphinx.pocketsphinx import *
from sphinxbase.sphinxbase import *

import os
import pyaudio
import wave
import audioop
from collections import deque
import queue
import time
import math

In [2]:
MODELDIR = "./models/"
DATADIR = "../corpus/"

In [4]:
# Get a default configuration that can be set with our preferred models
config = Decoder.default_config()

In [5]:
config.set_string('-hmm', os.path.join(MODELDIR, 'en-us/en-us'))
config.set_string('-lm', os.path.join(MODELDIR, 'en-us/en-us.lm.bin'))
config.set_string('-dict', os.path.join(MODELDIR, 'en-us/cmudict-en-us.dict'))

In [6]:
# Make a new decoder with the given models
decoder = Decoder(config)

In [11]:
CHUNK = 1024  # chunks of bytes to read each time from mic
FORMAT = pyaudio.paInt16 # expecting int16 audio format
CHANNELS = 1
RATE = 16000
SILENCE_LIMIT = 1  # Silence limit in seconds. The max ammount of seconds where
                           # only silence is recorded. When this time passes the
                           # recording finishes and the file is decoded

PREV_AUDIO = 0.5  # Previous audio (in seconds) to prepend. When noise
                          # is detected, how much of previously recorded audio is
                          # prepended. This helps to prevent chopping the beginning
                          # of the phrase.

THRESHOLD = 4500
num_phrases = -1
num_samples = 50

In [8]:
print("Getting intensity values from mic.")
p = pyaudio.PyAudio()

Getting intensity values from mic.


In [24]:
# See PyAudio Docs for API: https://people.csail.mit.edu/hubert/pyaudio/docs/
default_dev = p.get_default_input_device_info()
num_devs = p.get_device_count()
print(num_devs," devices")
for i in range(num_devs):
    print(i, p.get_device_info_by_index(i),'\n')


11  devices
0 {'defaultHighInputLatency': 0.01310657596371882, 'hostApi': 0, 'defaultLowOutputLatency': 0.01, 'maxOutputChannels': 0, 'structVersion': 2, 'defaultLowInputLatency': 0.0029478458049886623, 'index': 0, 'maxInputChannels': 2, 'defaultHighOutputLatency': 0.1, 'defaultSampleRate': 44100.0, 'name': 'Built-in Microphone'} 

1 {'defaultHighInputLatency': 0.1, 'hostApi': 0, 'defaultLowOutputLatency': 0.012607709750566893, 'maxOutputChannels': 2, 'structVersion': 2, 'defaultLowInputLatency': 0.01, 'index': 1, 'maxInputChannels': 0, 'defaultHighOutputLatency': 0.022766439909297054, 'defaultSampleRate': 44100.0, 'name': 'Built-in Output'} 

2 {'defaultHighInputLatency': 0.1, 'hostApi': 0, 'defaultLowOutputLatency': 0.0013333333333333333, 'maxOutputChannels': 2, 'structVersion': 2, 'defaultLowInputLatency': 0.01, 'index': 2, 'maxInputChannels': 2, 'defaultHighOutputLatency': 0.010666666666666666, 'defaultSampleRate': 48000.0, 'name': 'Soundflower (2ch)'} 

3 {'defaultHighInputLatency

In [32]:
# Create a thread-safe buffer of audio data
_buff = queue.Queue()

def callback(in_data, frame_count, time_info, status_flags):
    """Continuously collect data from the audio stream, into the buffer."""
    _buff.put(in_data)
    return (None, pyaudio.paContinue)


In [36]:
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                input_device_index=0,
                frames_per_buffer=CHUNK,
                stream_callback=callback)

In [14]:
if r < 3000:
    THRESHOLD = 3500
else:
    THRESHOLD = r + 100

NameError: name 'r' is not defined